# Installation on Google Collab
To install `QMCTorch` copy the code from one of the 3 text cells below in a code cell and run that cell. 

## Install QMCTorch from Pypi Package manager

```
! pip install qmctorch
```



## Install QMCTorch from GitHub
```
from google.colab import drive
drive.mount('/content/gdrive')
% cd gdrive/My Drive/
! git clone https://github.com/NLESC-JCER/QMCTorch
% cd QMCTorch
! pip install -e .
% cd ../
```

## Pull latest code from Github
```
from google.colab import drive
drive.mount('/content/gdrive')
% cd gdrive/My Drive/QMCTorch
! git pull origin master
! pip install -e .
% cd ../
```

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# % cd gdrive/My Drive/QMCTorch
# ! git pull origin master
# ! pip install -e .
# % cd ..

# Using QMCTorch

In [ ]:
import torch
from torch import optim
from qmctorch.scf import Molecule
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.solver import Solver
from qmctorch.sampler import Metropolis
from qmctorch.utils import set_torch_double_precision
from qmctorch.utils import plot_energy, plot_data

In [ ]:
use_gpu = torch.cuda.is_available()
set_torch_double_precision()

In [ ]:
mol = Molecule(atom='H 0 0 0.69; H 0 0 -0.69', unit='bohr', \
               calculator='pyscf', basis='sto-3g')

In [ ]:
wf = SlaterJastrow(mol, configs='cas(2,2)', cuda=use_gpu)

In [ ]:
sampler = Metropolis(nwalkers=2000, nstep=2000, step_size=0.2, \
                     ntherm=-1, ndecor=100, nelec=wf.nelec, \
                     init=mol.domain('atomic'), \
                     move={'type':'all-elec', 'proba':'normal'},
                     cuda=use_gpu)

In [ ]:
lr_dict = [{'params': wf.jastrow.parameters(), 'lr': 3E-3},
           {'params': wf.ao.parameters(), 'lr': 1E-6},
           {'params': wf.mo.parameters(), 'lr': 1E-3},
           {'params': wf.fc.parameters(), 'lr': 2E-3}]
opt = optim.Adam(lr_dict, lr=1E-3)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.90)

In [ ]:
solver = Solver(wf=wf, sampler=sampler,
                       optimizer=opt, scheduler=scheduler)

In [ ]:
obs = solver.single_point()

In [ ]:
solver.configure(freeze=['ao', 'mo'], loss='energy', grad='manual')
solver.track_observable(['local_energy'])

solver.configure_resampling(mode='update',
                            resample_every=1,
                            nstep_update=50)
solver.ortho_mo = False

In [ ]:
obs = solver.run(50)